In [1]:
%load_ext autoreload
%autoreload 2

In [211]:
from wdtools import *
import fiona
import webbrowser

In [3]:
revpath = inpath + '\GIS\ArcGIS Pro Project\DataReview\DataReview.gdb'

In [11]:
setID = 'Set003'

In [5]:
nm_to_add = 2143

In [12]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [42]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [43]:
len(wd.wetdet_delin_number.unique())

433

In [14]:
matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')

In [137]:
mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]

In [16]:
partial = gpd.read_file(revpath, layer=f'{setID}_partial')

In [17]:
def review_mapped(setID):
    mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]
    for wID in mapped0:
        gdf = gpd.read_file(revpath, layer=wID)
        if 'wdID' in gdf.columns:
            if len(gdf.wdID.unique()) > 1:
                print(wID)

In [18]:
def revise_single_partial_file(wID):
    gdf = gpd.read_file(revpath, layer=wID)
    selcols = ['Shape_Length', 'Shape_Area']
    if 'wdID' in gdf.columns:
        df = gdf.copy()[selcols +  ['wdID']]
    else:
        df = gdf.copy()[selcols]
        df.loc[:,'wdID'] = wID.replace('_', '-')
    df.loc[:,'geometry'] = gdf.loc[:,'geometry']
    return df

In [19]:
def merge_single_partial_file(wIDlist):
    df = pd.DataFrame()
    for wID in wIDlist:
        df=pd.concat([df, revise_single_partial_file(wID)], ignore_index=True)
    gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')
    return gdf

In [237]:
def combine_matched_digitized(setID, editedIDs, nm_to_add):
    mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]
    matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')
    partial = gpd.read_file(revpath, layer=f'{setID}_partial')
    gdf = merge_single_partial_file(mapped0)
    dat = gdf.append(partial, ignore_index=True)
    edited_gdf = matched[matched.wdID.isin(editedIDs)]
    edited_gdf = edited_gdf[['wdID', 'geometry']].dissolve('wdID')
    edited_gdf.loc[:, 'wdID'] = edited_gdf.index
    data1 = edited_gdf.append(dat[['wdID', 'geometry']], ignore_index=True)
    matched_df = matched[['wdID', 'geometry']].dissolve('wdID')
    matched_df.loc[:, 'wdID'] = matched_df.index
    excluded = [wdID for wdID in data1.wdID.unique() if wdID in matched_df.wdID.unique()]
    issues = pd.read_csv(os.path.join(inpath, "output", "to_review", f"{setID}_Mapping_Issues.csv"))
    issueIDs = list(issues.wetdet_delin_number.unique())
    matched_gdf = matched_df[~matched_df.wdID.isin(excluded+issueIDs)]
    data2 = matched_gdf.append(data1, ignore_index=True)
    wo_lot = gpd.read_file(revpath, layer=f'{setID}_wo_lot')
    wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)
    data3 = data2.append(wo_lot[['wdID', 'geometry']], ignore_index=True)
    unmatchedIDs = [wdID for wdID in wd.wetdet_delin_number.unique() if wdID not in data3.wdID.unique()]
    toCheck = [ID for ID in unmatchedIDs if ID not in issueIDs]
    return data3, toCheck

In [20]:
revise_single_partial_file('WD2019_0130')

,Shape_Length,Shape_Area,wdID,geometry
0,14604.540094,8.568226e+06,WD2019-0130,"MULTIPOLYGON (((710392.945 1475437.580, 710804..."


In [173]:
edited = ['WD2018-0261', 'WD2018-0422', 'WD2018-0635', 'WD2018-0642', 'WD2019-0048', 
          'WD2019-0369', 'WD2020-0507', 'WD2021-0536', 'WD2021-0590', 'WD2021-0700']

In [238]:
res = combine_matched_digitized(setID=setID, editedIDs=edited, nm_to_add=nm_to_add)

In [239]:
gdf = res[0]

In [240]:
len(gdf.wdID.unique())

424

In [241]:
res[1]

[]

In [199]:
unmatchedIDs = [wdID for wdID in wd.wetdet_delin_number.unique() if wdID not in gdf.wdID.unique()]

In [204]:
issueIDs = res[1]

In [219]:
toCheck = [ID for ID in unmatchedIDs if ID not in issueIDs]
print(toCheck)

['WD2018-0485', 'WD2019-0060', 'WD2021-0181']


In [227]:
wo_lot = gpd.read_file(revpath, layer=f'{setID}_wo_lot')

In [230]:
wd.loc[wd.record_ID.isin(wo_lot.record_ID.values), ['record_ID', 'wetdet_delin_number']]

,record_ID,wetdet_delin_number
83,2227,WD2020-0009
199,2343,WD2017-0124
217,2361,WD2018-0196


In [223]:
df = wd[wd.wetdet_delin_number.isin(toCheck)]

In [224]:
df = reindex_data(df)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [225]:
df.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,site_id,record_ID,notes,recyear,IDyear,missinglot,lots,lot,cnt_code,ORTaxlot
268,WD2018-0485,37S14E03,ROW,ROW Highway 504 and ROW of Henwas St.,Bly,Klamath,Bly Water Line Project,Development of new buried water line in road r...,43.390332,-121.044816,...,110223,2412,ROW,2018,2018,Y,ROADS,ROADS,18,1837.00S14.00E0300--0000ROADS
269,WD2018-0485,37S14E03DB,ROW,ROW Highway 504 and ROW of Henwas St.,Bly,Klamath,Bly Water Line Project,Development of new buried water line in road r...,43.390332,-121.044816,...,141481,2413,ROW,2018,2018,Y,ROADS,ROADS,18,1837.00S14.00E03DB--0000ROADS
279,WD2019-0060,37S14E03DB,ROW,"Along Elder and Gerber Streets, S of Henwas St",Bly,Klamath,Bly New Waterline,new waterline,42.385614,-121.049808,...,127769,2423,ROW,2019,2019,Y,ROADS,ROADS,18,1837.00S14.00E03DB--0000ROADS


In [226]:
df.ORTaxlot.unique()

array(['1837.00S14.00E0300--0000ROADS', '1837.00S14.00E03DB--0000ROADS',
       '1837.00S14.00E03CD--0000ROADS', '1837.00S14.00E03CA--0000ROADS',
       '2903.00N10.00W32A0--000054069'], dtype=object)

In [ ]:
wdID = 'WD2021-0181'
url = wd.loc[wd.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
webbrowser.open(url)

In [216]:
url

nan

In [221]:
wdID

'WD2021-0181'

In [222]:
wd.loc[wd.wetdet_delin_number == wdID, 'record_ID']

659    2803
Name: record_ID, dtype: int32

In [ ]:
extract_page_from_docLink(url=url, pageNm=4, wdID=wdID)

In [214]:
extract_page_from_locPath(filePath=r'C:\Users\clid1852\Downloads\WD20190060 AgencyDecision.pdf', #T:\DCProjects\EPA-WD\docs\
                          pageNm=8, wdID=wdID)

In [47]:
tomapID = matched[matched.notes.str.contains('Partial', case=False, na=False)].wdID.unique()